In [166]:
from happytransformer import HappyTextClassification
from dataclasses import dataclass
from typing import Callable
from pathlib import Path
from newsapi import NewsApiClient
from datetime import date
from newspaper import Article
import nltk

nltk.data.path = [str(Path().resolve().parent.joinpath("nltk_data"))]

# nltk.download("punkt", download_dir="./nltk_data")

@dataclass
class Digest:
    html: str
    authors: str
    publish_date: str
    keywords: str
    summary: str
    title: str
    text: str

def process_article(url: str) -> Callable:
    article = Article(url)
    def _process_article() -> Digest:
        article.download()
        article.parse()
        article.nlp()
        return Digest(
            html=article.html,
            authors=article.authors,
            publish_date=article.publish_date,
            keywords=article.keywords,
            summary=article.summary,
            title=article.title,
            text=article.text,
        )
    return _process_article

classifier = HappyTextClassification(
    model_type="DISTILBERT", num_labels=2,
    model_name="distilbert-base-uncased-finetuned-sst-2-english")


06/22/2022 23:48:54 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [167]:
# Init pass 9gPj8KZ8XJsLbcT
api = NewsApiClient(api_key='98d763c2280d4820913c4abf3ff0270b')

# /v2/everything
all_articles = api.get_everything(q='bitcoin',
                                  # sources='bbc-news,the-verge,Bloomberg',
                                  # category='business',
                                  from_param=date.today(),
                                  language='en')

# /v2/top-headlines/sources
# sources = api.get_sources()

all_articles["totalResults"]

420

In [168]:
articles = list([{"title":article["title"], "url":article["url"]} for article in all_articles["articles"]])

digest: Digest = process_article(articles[0]["url"])()

result = classifier.classify_text(digest.summary)

result

TextClassificationResult(label='NEGATIVE', score=0.9898502826690674)

In [169]:
digest.summary

'FTX\'s Sam Bankman-Fried has provided a $250 million loan to bail out crypto lender BlockFi.\nThe crypto billionaire also loaned broker Voyager Digital $485 million in cash and bitcoin last week.\n"Today we\'re injecting $250 million into BlockFi and partnering with them so they can navigate the market from a position of strength," Bankman-Fried announced Tuesday.\nMajor lender Celsius announced it was freezing all withdrawals, and BlockFi and Voyager have both suffered significant losses as major cryptocurrencies plunged.\n"Sam Bankman-Fried is the new John Pierpont Morgan," Scaramucci told Bloomberg.'